# Example on loading probabilities output from the cram file

For installation use the following pip install, required python 3.11



In [2]:
# pip install ugbio_core[vcfbed]

In [3]:
import pysam
import numpy as np
import itertools

import ugbio_core.flow_format.flow_based_read as fbr
from ugbio_core.flow_format.error_model import key2base 



## Load cram file 

This load the sam file and pull the probabilites 

In [4]:
# !aws s3 cp s3://ultimagen-workflow-resources-us-east-1/test_data/cram_example/cram_example.cram /data/Runs/examples/cram_example.cram

cram_file = '/data/Runs/examples/cram_example.cram'
pysam.index(cram_file)
alignment_file = pysam.AlignmentFile(cram_file, 'r')
prob_i = np.zeros((1000, 436, 21))
sequences = []

for i, record in enumerate(itertools.islice(alignment_file,0,1000)):
    # Process each record
        flow_read = fbr.FlowBasedRead.from_sam_record(record, max_hmer_size=20)
        L = flow_read._flow_matrix.shape[1]
        prob_i[i,:L] = flow_read._flow_matrix.transpose(1, 0)

        sequences.append(record.seq)

## Validate the data created

In [4]:
i = 10

print('sequence of the first 100 bases of read number ', i)
print( sequences[i][:100] )

# load probability vector of the i'th read

probability_flow_i = prob_i[i]

# convert probability vector to hmmers
called_hmers = probability_flow_i.argmax(axis=-1)

# convert hmmers to bases
called_bases = key2base(called_hmers)

print('called bases of the first 100 bases of read number ', i)
print( ''.join(called_bases[:100]) )

sequence of the first 100 bases of read number  10
CAAAGATAAATGAGCAGGCAAGCTGGCTAGAAAACCACTAAGGGTATTTATTCTTTAAAGAATCTTTACAGGGTCAAAGAAGAATGGGTCTTAACTGGCT
called bases of the first 100 bases of read number  10
AAAGATAAATGAGCAGGCAAGCTGGCTAGAAAACCACTAAGGGTATTTATTCTTTAAAGAATCTTTACAGGGTCAAAGAAGAATGGGTCTTAACTGGCTA


There is typical issue typical of the first cycle in the transformation, miss correct probability in the first cycle.

